In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys, os
sys.path.append(os.path.abspath(".."))

from pycfx.datasets import *
from pycfx.models import *
from pycfx.benchmarker import *
from pycfx.helpers.visualisation import *
from pycfx.conformal import *
from pycfx.counterfactual_explanations import *

from datetime import datetime

print(f"Start {datetime.now()}")


dataset_cls = CaliforniaHousing
mlp_config = {"epochs": 100, "batch_size": 64}
# mlp_config = {"epochs": 50, "batch_size": 256}

is_rf = False

print(f"Start {datetime.now()}")
mlp_config = {"epochs": 100, "batch_size": 64}

dataset = dataset_cls(0.6, 0.2, 0.2)
model_factories = []

if is_rf:
    factory = ModelFactory(RandomForestSKLearn, dataset.input_properties, config={}, config_multi={})
    model_factories.append(factory)
else:
    factory = ModelFactory(PyTorchMLP, dataset.input_properties, config=mlp_config, config_multi={})
    model_factories.append(factory)

n_factuals_main = 100
n_repeats = 2
path = Path("results_path")
use_pretrained = True

metrics = [
    FailuresMetric(), 
    DistanceMetric(), 
    ValidityMetric(),  
    ImplausibilityMetric(included_prop=0.1), 
    ImplausibilityMetric(included_prop=1),
    LOFMetric(n_neighbours=20, stratified=True),
    SensitivityMetric(n_sensitivity=25, n_neighbours=4, budget=0.001), 
    StabilityMetric(n_neighbours=8, budget=0.001),
]

conformal_config = {
    "alpha": [0.01, 0.05, 0.1], "scorefn_name": ["linear_normalised" if is_rf else "linear2"], "kernel_bandwidth": [0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45], 
}

generators = [
    GeneratorFactory([MinDistanceCF], config={}, config_multi={}),

    GeneratorFactory([ConformalCF], config={"conformal_class": SplitConformalPrediction}, config_multi={"conformal_config": {"alpha": [0.01, 0.05, 0.1]}}),

    GeneratorFactory([ConformalCF], config={"conformal_class": ConformalCONFEXTree}, config_multi={
        "conformal_config": conformal_config
    }),
]

if is_rf:
    generators.extend([
        GeneratorFactory([FOCUSGenerator], config={"n_iter": 200}, config_multi={}),
        GeneratorFactory([FeatureTweakGenerator], config={"epsilon": 0.01}, config_multi={})
    ])
else:
    f1 = GeneratorFactory([WachterGenerator], config={"mad": True, "new": 3}, config_multi={})
    f2 = GeneratorFactory([SchutGenerator], config={"new": True, "new2": 3}, config_multi={})
    f3 = GeneratorFactory([ECCCOGenerator], config={"new": 3}, config_multi={"conformal_config": {"alpha": [0.01, 0.05, 0.1]}})
    generators.extend([f1, f2, f3])
    

## Do not modify below
print("Initializing CFBenchmarker...")
benchmarker = CFBenchmarker(dataset, n_factuals_main, n_repeats, metrics, model_factories, generators, path, use_pretrained=use_pretrained)

print("Setting up models...")
benchmarker.setup_models()

print("Evaluating models...")
benchmarker.evaluate_models()

print("Setting factuals...")
benchmarker.set_factuals()

print("Initializing generators...")
benchmarker.initialise_generators()

print("Generating counterfactuals...")
# benchmarker.get_counterfactuals(reset=not use_pretrained)

print("Evaluating counterfactuals...")
# df_out = benchmarker.evaluate_counterfactuals()


print("Test conformal...")
benchmarker.test_conformal()


print(f"Evaluation complete. See {path}")
print(f"End {datetime.now()}")



In [ ]:
structure = ["MinDist", "Wachter", "Greedy", "ConfexNaive", "ECCCo", "ConfexTree"]
indent_map = {"ConfexNaive": ["\\alpha"], "ECCCo": ["\\alpha"], "ConfexTree": ["\\alpha", "\\text{bw}"]}
format_map = {"Wachter": "WachterGenerator", "Greedy": "SchutGenerator", "MinDist": "MinDistanceCF", "ECCCo": "ECCCOGenerator", "ConfexNaive": "SplitConformalPrediction", "ConfexTree": "ConformalCONFEXTree", "\\alpha": "alpha", "\\text{bw}": "kernel_bandwidth"}
formatted_col_names = ["Distance", "Plausibility", "Implausibility", "Sensitivity $(10^{-1})$", "Stability"]

In [ ]:
benchmarker.generate_table(formatted_col_names, structure, indent_map, format_map, scaling=[(3, 0.1)], dp2=True)

In [ ]:
benchmarker.generate_conformal_sim_plot()

In [ ]:
benchmarker.get_alpha_bandwidth_plots()

In [ ]:
structure = ["ConfexNaive", "ConfexTree"]
indent_map = {"ConfexNaive": ["\\alpha"], "ConfexTree": ["\\alpha", "\\text{bw}"]}
format_map = {"ConfexNaive": "SplitConformalPrediction", "ConfexTree": "ConformalCONFEXTree", "\\alpha": "alpha", "\\text{bw}": "kernel_bandwidth"}

In [ ]:
rc = benchmarker.generate_conformal_table(structure, indent_map, format_map)